## Update 25/11

1. Using 15*TTA


In [ ]:
## Importing Required Libaries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import glob
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow import keras

SEED = 42
DEBUG = False


## Loading Best Model

In [ ]:
weight_path = '../input/efficientnetb3-keras-tf2-baseline-training/best_model.hdf5'

my_model = load_model(weight_path)



**While Saving I am saving weights plus architecture so I don't need to build the model again here**

## Test Data Generator

In [ ]:

test_images = glob.glob('../input/cassava-leaf-disease-classification/test_images/*.jpg')

df_test = pd.DataFrame(test_images, columns = ['path'])



def make_test_gen( batch_size=64):
    my_test_idg = ImageDataGenerator(#rescale=1. / 255.0,
                                    horizontal_flip = True, 
                                    vertical_flip = True, 
                                    height_shift_range=0.2, 
                                    width_shift_range=0.2, 
                                    brightness_range=[0.7, 1.5],
                                    rotation_range=30, 
                                    shear_range=0.2,
                                    fill_mode='nearest',
                                    zoom_range=[0.3,0.6])
        
    test_gen= my_test_idg.flow_from_dataframe(dataframe=df_test,

                                                x_col="path",
                                                y_col=None,
                                                batch_size=batch_size,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None,
                                                target_size=(300,300)) ## (height, width)
    return test_gen





In [ ]:
test_gen = make_test_gen(batch_size = 128)


In [ ]:
preds = []
for i in range(15):
    #test_gen.reset()
    local_pred = my_model.predict(test_gen,  verbose = True)
    preds.append(local_pred)

In [ ]:
pred_test = np.mean(preds, axis=0)

## Generating Predictions

In [ ]:
#pred_test = my_model.predict(test_gen,  verbose = True)
pred_test_labels = np.argmax(pred_test, axis = -1)

## Generating Predictions File

In [ ]:
final_submission = df_test
final_submission['image_id'] = final_submission.path.str.split('/').str[-1]
final_submission['label'] = pred_test_labels

final_csv = final_submission[['image_id', 'label']]
final_csv.head()

In [ ]:
final_csv.to_csv('submission.csv', index=False)

## Train Baseline Kernel is [here](https://www.kaggle.com/harveenchadha/efficientnetb3-keras-tf2-baseline-training).

